In [1]:

%pprint
import warnings

warnings.filterwarnings('ignore')

Pretty printing has been turned OFF



----
# Find Basic Qualifications Section in a File

In [2]:

%%time
%matplotlib inline
%run ../py/chart_utls.py
%run ../load_magic/charts.py
import random

qu = ChartUtilities()
dir()

Wall time: 1.37 s


['ChartUtilities', 'FACEBOOK_ASPECT_RATIO', 'HTML', 'In', 'Out', 'TWITTER_ASPECT_RATIO', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'ball_and_chain', 'colormaps_list', 'cycler', 'cypher_utlis', 'display', 'exit', 'first_order_linear_scatterplot', 'get_color_cycler', 'get_inf_nan_mask', 'get_ipython', 'html_analysis', 'np', 'os', 'pd', 'pearsonr', 'plt', 'qu', 'quit', 'r', 'random', 'save_fig_as_various', 'set_matplotlib_formats', 'sns', 'storage', 'to_hex', 'warnings']

In [3]:

cypher_str = '''
    MATCH (pos:PartsOfSpeech {is_header: 'True'})
    RETURN
        pos.pos_symbol as pos_symbol,
        pos.pos_explanation as pos_explanation,
        pos.is_task_scope as is_task_scope,
        pos.is_minimum_qualification as is_minimum_qualification,
        pos.is_preferred_qualification as is_preferred_qualification,
        pos.is_legal_notification as is_legal_notification,
        pos.is_job_title as is_job_title,
        pos.is_office_location as is_office_location,
        pos.is_job_duration as is_job_duration,
        pos.is_supplemental_pay as is_supplemental_pay,
        pos.is_educational_requirement as is_educational_requirement,
        pos.is_interview_procedure as is_interview_procedure,
        pos.is_corporate_scope as is_corporate_scope,
        pos.is_posting_date as is_posting_date,
        pos.is_other as is_other;'''
pos_df = pd.DataFrame(qu.cu.get_execution_results(cypher_str, verbose=False))
color_cycler = get_color_cycler(pos_df.shape[0])
header_pos_list = pos_df.pos_symbol.tolist()
rgba_dict = {}
for header_pos_symbol, face_color_dict in zip(header_pos_list, color_cycler()):
    face_color = face_color_dict['color']
    rgba_dict[header_pos_symbol] = face_color

In [4]:

for header_pos_symbol, face_color in rgba_dict.copy().items():
    nonheader_pos_symbol = 'O-' + header_pos_symbol.split('-')[1]
    fc_array = face_color.copy()
    fc_array[-1] = 0.5
    rgba_dict[nonheader_pos_symbol] = fc_array

In [5]:

from matplotlib.colors import to_hex
from IPython.display import HTML, display

html_str = ''
for pos, rgba in rgba_dict.items():
    hex_str = to_hex(rgba, keep_alpha=True)
    html_str += f'<span style="color:{hex_str};">{pos}</span>'
    if pos == 'H-O':
        html_str += '<br />'
    else:
        html_str += '&nbsp;'
display(HTML(html_str))

In [6]:

%%time
HEADER_PATTERN_DICT = qu.s.load_object('HEADER_PATTERN_DICT')
file_name = random.choice(list(HEADER_PATTERN_DICT.keys())).strip()
qu.visualize_cfr_child_str_predictions(file_name=file_name)

Wall time: 64.9 ms


['O', 'H', 'O', 'O', 'O', 'O', 'O', 'O-TS', 'H', 'O', 'O', 'O', 'O', 'H', 'O', 'O-RQ', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-SP', 'H-SP', 'O-SP', 'O', 'O', 'O-SP', 'O-SP', 'O', 'O', 'O', 'O', 'H-JD', 'O-OL', 'H-OL', 'O', 'H-CS', 'O', 'O', 'O', 'H-SP', 'O', 'H-OL', 'O', 'H-IP', 'O', 'O']

In [7]:

%run ../py/cypher_utlis.py
cu = CypherUtilities()

In [8]:

row_objs_list = cu.get_execution_results(cu.select_filename_isheader_cypher_str, verbose=False)
is_header_df = pd.DataFrame(row_objs_list)
def f(df):
    return df.is_header.str.match('(True|False)').all(skipna=False)
s = is_header_df.groupby('file_name').apply(f)
file_names_list = s[s==True].index.tolist()

In [13]:

import random

file_name = random.choice(file_names_list)
NAVIGABLE_PARENT_IS_HEADER_DICT = cu.s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
child_strs_list = cu.get_child_strs_from_file(file_name)

In [10]:

%run ../py/html_analysis.py
hc = HeaderCategories()
crf = CrfUtilities()
feature_dict_list = HEADER_PATTERN_DICT[file_name]
feature_tuple_list = [hc.get_feature_tuple(feature_dict) for feature_dict in feature_dict_list]
prediction_dict_list = crf.CRF.predict_marginals_single(crf.sent2features(feature_tuple_list))

In [11]:
for prediction_dict, child_str in zip(prediction_dict_list, child_strs_list):
    print(child_str, prediction_dict['H-RQ'], prediction_dict['H-PQ'])

The Data Science Transformation team seeks a hands-on Manager/Principal Machine Learning Data Scientist to join our team. You should be an exceptional disciplined and self-motivated Data Scientist with predictive modeling and machine learning advanced analytics experience and a passion for working with healthcare data (relevant experience with data from other industries of interest as well; financial, retail, telco, etc), experience using various computational approaches, and a portfolio of projects you can show and talk about. 0.03438742620665903 0.033299449976003634
<b>Primary Responsibilities:</b> 0.02828884261457956 0.026402727370210936
<li>A leader. You're a hands-on Machine Learning leader who challenge conventional thinking and work with stakeholders to identify and improve the status quo. You i ndependently identify significant opportunities in an ambiguous area and builds consensus around roadmaps and how to evaluate success.</li> 0.02689934325127304 0.02436782265332839
<li>Pe

In [12]:

ea = ElementAnalysis()
consecutives_list, pos_list = cu.find_basic_quals_section(child_strs_list, hc=hc, ea=ea)
print(consecutives_list, pos_list)
rq_idx_list = ea.get_idx_list(pos_list, 'H')

# Display the Requirements sections in their own HTML
if len(rq_idx_list):
    consecutives_idx_list = ea.get_idx_list(consecutives_list, ('H', 1))
    for rq_idx, consecutives_idx in zip(rq_idx_list, consecutives_idx_list):
        o_count = consecutives_list[consecutives_idx+1][1]
        display(HTML(''.join(child_strs_list[rq_idx:rq_idx+o_count+1])))

[('H', 1)] ['H']


IndexError: list index out of range